In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SELECTED_ANNOTATOR=['Nathan','Huixue']

In [ ]:
DIR_PATH='/content/drive/MyDrive/CAM_NER_RE_files/p2175_CAM_Gene_v5'

In [ ]:
SELECT_ABSTRACTS=['10072988','10172712','10380449','11191042','11301211',
                  '11677409','11986575','12434945','12584510','12799612',
                  '12828853','12853717','14521273','14605483','15039508',
                  '15354371','15804591','15845676','16078356','16388436',
                  '16459202','16528927','16643478','16679985','16715021']

In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from abc import ABC, abstractmethod
import xmltodict
import os
import json
from collections import defaultdict

In [ ]:
interested_tags2=[]

class AnnotationsExtraction(ABC):
  DATA_SET_DIR=''

  def __init__(self, file_path):
    self.file_path=file_path

  def get_text(self):
    file = open(self.file_path, 'r', encoding='utf-8').read()
    #file = open(path, 'r',encoding='windows-1252').read() #encoding='windows-1252'
    my_dict = xmltodict.parse(file)
    my_dict = json.loads(json.dumps(my_dict))
    try:
        title= my_dict['collection']['document']['passage'][0]['text']
    except:
        try:
            title = my_dict['collection']['document']['passage']['text']
        except:
            title=''

    try:
        abstract=my_dict['collection']['document']['passage'][1]['text']

    except:
        abstract=''

    passage= my_dict['collection']['document']['passage']
    id=my_dict['collection']['document']['id']

    text=title+' '+abstract

    return text,passage,id

  def get_annotation(self,edit_annotation=False):
      passage_list=[]
      _,passages,_= self.get_text()
      try:
          if isinstance(passages,list):
              passage_list = passages
          elif isinstance(passages,dict):
              passage_list = [passages]
      except Exception as e:
          print(e)

      annodation_dic={}
      annotation_list=[]

      if passage_list!=[]:
        annotations=[]
        for p in passage_list:
            try:
                annotations = p['annotation']
            except Exception as e:
                #print(">>>>>> error:",e,'passage:',p)
                pass

            anno_list=[]
            try:
                if isinstance(annotations, list):
                    anno_list = annotations
                elif isinstance(annotations, dict):
                    anno_list = [annotations]
            except Exception as e:
                print(e)
                # raise ValidationError(_(str(e)))
            for i in anno_list:
                annotation_list.append(i)

      if annotation_list!=[]:
        for anno in annotation_list:
            annotation_id = anno['@id']
            location_stat = anno['location']['@offset']
            location_end = int(anno['location']['@offset']) + int(anno['location']['@length'])
            location_length = anno['location']['@length']
            type = 'None'
            identifier = 'None'
            annotator = 'None'
            txt=anno['text']
            for j in anno['infon']:
                try:
                  if j['@key'] == 'type':
                      type = j['#text']
                except:
                  pass
                try:
                  if j['@key'] == 'identifier' and '#text' in j.keys():
                      identifier = j['#text']
                except:
                  pass
                try:
                  if j['@key'] == 'annotator':
                      annotator = j['#text']
                except:
                  pass

            if identifier!= None and identifier in interested_tags2:
                   type=identifier

            if edit_annotation:
                if type in interested_tags2:
                    annodation_dic[annotation_id] = {'type': type, 'identifier': identifier, 'annotator': annotator,
                                                     'start': location_stat,
                                                     'end': location_end, 'length': location_length, 'txt': txt}
            else:
                #if type in interested_tags2:
                if type !='None':
                  annodation_dic[annotation_id] = {'type': type, 'identifier': identifier, 'annotator': annotator,
                                                     'start': location_stat,
                                                     'end': location_end, 'length': location_length, 'txt': txt}
      return annodation_dic

  def get_relation(self, include_anno_type=False,select_annotator=True,annotator=SELECTED_ANNOTATOR[0]):
      _,passages,id= self.get_text()
      annotation_dict=self.get_annotation()
      passage_list=[]
      try:
          if isinstance(passages,list):
            passage_list = passages
          elif isinstance(passages,dict):
            passage_list = [passages]
      except Exception as e:
        print(e)

      relation_dic=defaultdict(dict)
      relation_list=[]
      all_relation_list_include_anno=[]
      all_relation_list=[]
      relation_dic={}

      if passage_list!=[]:
          annotations=[]
          for p in passage_list:
              try:
                  annotations = p['relation']
              except Exception as e:
                #print(">>>>>> error:",e,'passage:',p)
                  pass

              anno_list=[]
              try:
                  if isinstance(annotations, list):
                    anno_list = annotations
                  elif isinstance(annotations, dict):
                    anno_list = [annotations]
              except Exception as e:
                print(e)
                # raise ValidationError(_(str(e)))
              for i in anno_list:
                relation_list.append(i)

      if relation_list!=[]:
          #print("EXTRACTING relationships")
          relation_dic[id]={}
          for rela in relation_list:
            #print(" rela:", rela)
            relation_id=rela['@id']
            rela_annotator='None'
            relation_dic[id][relation_id]=[]
            rela_object=[]
            rela_subject=[]

            for j in rela['infon']:
              if j['@key']=='type':
                rela_type=j['#text']
              if j['@key']=='annotator':
                rela_annotator=j['#text']

            if select_annotator:
              if rela_annotator == annotator:
                rela_subject,rela_object=self.CombineRelaSubObj(rela,annotation_dict)
              else:
                pass

            if len(rela_object)>0 and len(rela_object)>0:
              for s in rela_subject:
                for o in rela_object:
                  relation_dic[id][relation_id].append({'relation':rela_type,'subject':s,'object':o})
                  all_relation_list.append([rela_type,s,o])
                  if include_anno_type:
                    try:
                      s_type=annotation_dict[s]['type']
                      o_type=annotation_dict[o]['type']
                    except:
                      print("please check if you include the annotation_dict")
                    all_relation_list_include_anno.append([rela_type,s,o,s_type,o_type])

      return relation_dic,all_relation_list,all_relation_list_include_anno

  def CombineRelaSubObj(self,rela,annotation_dict):
    rela_object=[]
    rela_subject=[]
    for n in rela['node']:
      try:
        if n['@role']=='object':
          rela_object.append(n['@refid'])
      except:
        pass
      try:
        if n['@role']=='subject':
          rela_subject.append(n['@refid'])
      except:
        pass
    return rela_subject,rela_object

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import re

# --- Subject & object markup ---
SUB_START_CHAR = "["
SUB_END_CHAR = "]"
OBJ_START_CHAR = "{"
OBJ_END_CHAR = "}"

class AbstractProcessing:
  def __init__(self, file_path,annotator):
    self.file_path=file_path
    self.annotator=annotator


  def extractor(self):
    extractor=AnnotationsExtraction(self.file_path)
    text,_,id=extractor.get_text()
    annodation_dic=extractor.get_annotation()
    relation_dic,_,_=extractor.get_relation(annotator=self.annotator)

    return id,text,annodation_dic,relation_dic

  def SentToken(self):
    id,text,_,_=self.extractor()
    splited_text=sent_tokenize(text)
    #Spans = list(pt().span_tokenize(text))
    SentsSpan=[]

    #total_len=0
    sent_start=0
    sent_end=0
    sent_len=0

    for sent_id,i in enumerate(splited_text):
      if sent_id!=0:
        sent_start=sent_end+1
      sent_len=len(i)
      sent_end=sent_start+sent_len
    #total_len+=len(i)
      SentsSpan.append([sent_start,sent_end])
    return splited_text,SentsSpan

  def ProcessingPositiveSent(self):
    PositiveSents=defaultdict(dict)
    id,text,annodation_dic,relation_dic=self.extractor()
    splited_text,SentsSpan=self.SentToken()

    re_id=0
    if relation_dic!={}:
      for re_list_id in relation_dic[id]:
        for r in relation_dic[id][re_list_id]:
          subject_id=r['subject']
          object_id=r['object']
          label=r['relation']
          #print(r)
          subject_anno=annodation_dic[subject_id]
          object_anno=annodation_dic[object_id]
          #print(subject_anno,object_anno)

          for sent_id,sent_span in enumerate(SentsSpan):
            if int(subject_anno['start'])>=sent_span[0] and int(subject_anno['end'])<=sent_span[1] and\
              int(object_anno['start'])>=sent_span[0] and int(object_anno['end'])<=sent_span[1]:
              PositiveSents[id][re_list_id]=[]
              try:
                new_sent=str(splited_text[sent_id])
                new_sent=re.split(subject_anno['txt'],new_sent)
                new_sent=new_sent[0]+SUB_START_CHAR+subject_anno['txt']+SUB_END_CHAR+new_sent[1]
                new_sent=re.split(object_anno['txt'],new_sent)
                print(new_sent,object_anno['txt'],object_anno['start'],object_anno['end'],sent_span)
                print(splited_text[sent_id])
                new_sent=new_sent[0]+OBJ_START_CHAR+object_anno['txt']+OBJ_END_CHAR+new_sent[1]
                PositiveSents[id][re_list_id].append({'sentence':new_sent,'label':label,'subject':subject_anno,'object':object_anno})
                re_id+=1
              except:
                print(id,new_sent,object_anno['txt'],subject_anno['txt'])
                new_sent=str(splited_text[sent_id])
                PositiveSents[id][re_list_id].append({'sentence':new_sent,'label':label,'subject':subject_anno,'object':object_anno})
                re_id+=1

    #print(PositiveSents)

    return id,PositiveSents


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class CalculateAggrement(ABC):
  def __init__(self, annotation1,annotation2,restrict=False):
    self.annotation1=annotation1,
    self.annotation2=annotation2,
    self.restrict=restrict

  def compare_list(self,a,b):
    if_same=0
    for i in a:
        if i in b:
            if_same=1
        break
    return if_same

  def if_same_entity(self, an1, an2):
    """
    :param an1, an2: annotation by first and second anotator, an=[type,start,end]
    :param restrict: if yes, only if the type, start, and end of an1 and an2 be the same,
    will consider an1 and an2 as the same annotation
    :return: weather if an1 and an2 the same annotation
    """
    if_same = 0

    if self.restrict:
        if an1==an2:
            if_same=1
    else:
        type1=an1[0]
        type2=an2[0]

        start1=an1[1]
        start2=an2[1]

        end1=an1[2]
        end2=an2[2]

        duration1 = [i for i in range(int(start1), int(end1) + 1)]
        duration2 = [i for i in range(int(start2), int(end2) + 1)]

        if type1 == type2:
            if_same=self.compare_list(duration1,duration2)

    return if_same

  def if_same_relation(self,rela1,rela2):
    #print(rela1)
    if_same=False
    subject1=rela1['subject']
    subject2=rela2['subject']

    object1=rela1['object']
    object2=rela2['object']

    sunject1_pos=[subject1['type'],subject1['start'],subject1['end']]
    subject2_pos=[subject2['type'],subject2['start'],subject2['end']]

    if self.if_same_entity(sunject1_pos,subject2_pos):
      object1_pos=[object1['type'],object1['start'],object1['end']]
      object2_pos=[object2['type'],object2['start'],object2['end']]
      if self.if_same_entity(object1_pos,object2_pos):
        if rela1['label']==rela2['label']:
          if_same=True

    return if_same

  def cal_aggrement(self):
    """Computes Cohen kappa for pair-wise annotators.

                 Pr(a) - Pr(e)
    Cohen's 𝜅 = ---------------
                   1 - Pr(e)

    Where...
        Pr(a) = the percentage of observed agreement
        Pr(e) = the percentage of expected agreement.

    :param ann1: relation annotations provided by first annotator
    :type ann1: list
    :param ann2: relation annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    annotation1=[]

    for an1 in self.annotation1:
      if len(an1)!=0:
        for r1 in an1:
          annotation1.append(r1)

    annotation2=[]

    for an2 in self.annotation2:
      if len(an1)!=0:
        for r2 in an2:
          annotation1.append(r2)

    count = 0
    for an1 in self.annotation1:
        for an2 in self.annotation2:
          if len(an1)!=0 and len(an2)!=0:
            print("---------")
            print("an1:",len(an1),an1)
            print("an2:",len(an2),an2)
            for r1 in an1:
              for r2 in an2:
                if self.if_same_relation(r1,r2):
                  count += 1
    A = count / (len(self.annotation1)+1e-5)  # observed agreement A (Po)

    #for i in self.annotation1:
      #print(i)
    try:
        #annotation1 =[i[0] for i in  self.annotation1]
        type_list1=[i['label'] for i in annotation1]
        #annotation2 =[i[0] for i in  self.annotation2]
        type_list2=[i['label'] for i in annotation2]
        uniq = set(type_list1 + type_list2)
    except:
        #annotation1 =[i[0] for i in  self.annotation1]
        type_list1=[i['label'] for i in annotation1]
        type_list2=['None']
        uniq=set(type_list1)

    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = list(type_list1).count(item)
        cnt2 = list(type_list2).count(item)
        count = ((cnt1 / (len(type_list1)+1e-5)) * (cnt2 / (len(type_list2)+1e-5)))
        E += count

    if len(annotation1)==0 and len(annotation2)==0:
      print(annotation1)
      print(annotation2)
      return 1

    else:
      return round((A - E) / (1 - E + 1e-5), 4)

In [ ]:

anno1=[]
anno2=[]

files=os.listdir(DIR_PATH)

for f in files:
  file_path=os.path.join(DIR_PATH,f)
  extractor=AnnotationsExtraction(file_path)
  processor1=AbstractProcessing(file_path,SELECTED_ANNOTATOR[1])
  processor2=AbstractProcessing(file_path,SELECTED_ANNOTATOR[0])
  text,passage,id=extractor.get_text()
  if id in SELECT_ABSTRACTS:
    #relation_dic2,all_relation_list,all_relation_list_include_anno=extractor.get_relation(annotator=SELECTED_ANNOTATOR[0])
    #relation_dic1,all_relation_list,all_relation_list_include_anno=extractor.get_relation(annotator=SELECTED_ANNOTATOR[1])
    _,relation_dic2=processor2.ProcessingPositiveSent()
    _,relation_dic1=processor1.ProcessingPositiveSent()
    anno2.append(relation_dic2)
    anno1.append(relation_dic1)

In [ ]:
total_score=0
num=0

for a1 in anno1[1:]:
  for a2 in anno2[1:]:
    if a1.keys()==a2.keys() and len(a1.keys())!=0:
      relation1_list=[]
      relation2_list=[]

      for absract_id in a1.keys():
       for relatoion_id in a1[absract_id].keys():
         for r in a1[absract_id][relatoion_id]:
           if len(r)!=0:
            relation1_list.append(r)

      for absract_id in a2.keys():
       for relatoion_id in a2[absract_id].keys():
         for r in a2[absract_id][relatoion_id]:
           if len(r)!=0:
            relation2_list.append(r)

      print("\n","----------------calculate abstract: ",absract_id,"----------------")
      #print(len(relation1_list),relation1_list)
      Calculator=CalculateAggrement(relation1_list,relation2_list)
      aggrement_score=Calculator.cal_aggrement()
      total_score+=aggrement_score
      num+=1
      print("aggrement_score:",aggrement_score)
    elif a1.keys()==a2.keys() and len(a1.keys())==0 and len(a2.keys())==0:
      aggrement_score=1
      total_score+=aggrement_score
      num+=1


print("\n","average aggrement score:",total_score/num)



 ----------------calculate abstract:  12828853 ----------------
---------
an1: 3 [{'sentence': 'CONCLUSION: Attenuation of {ROS} production by neutrophils may play a role in the effects of [LLLT] in the treatment of inflammatory tissues.', 'label': 'ASSOCIATED_WITH', 'subject': {'type': 'Energy_therapies', 'identifier': 'None', 'annotator': 'Huixue', 'start': '1152', 'end': 1156, 'length': '4', 'txt': 'LLLT'}, 'object': {'type': 'molecular', 'identifier': 'None', 'annotator': 'Huixue', 'start': '1088', 'end': 1091, 'length': '3', 'txt': 'ROS'}}, {'sentence': 'There is a possible usage of [LLLT] to improve {wound healing} in smokers.', 'label': 'AFFECTS', 'subject': {'type': 'Energy_therapies', 'identifier': 'None', 'annotator': 'Huixue', 'start': '1228', 'end': 1232, 'length': '4', 'txt': 'LLLT'}, 'object': {'type': 'sign_symptom', 'identifier': 'None', 'annotator': 'Huixue', 'start': '1244', 'end': 1257, 'length': '13', 'txt': 'wound healing'}}, {'sentence': 'BACKGROUND DATA: [LLLT] 

In [ ]:
anno2

In [ ]:
anno1